# Análisis no lineal aproximado

## Introducción

El modelo de la teoría linealizada de la elasticidad tiene entre sus aproximaciones fundamentales la de asumir que no hay diferencias importantes entre la configuración original y la configuración deformada de una estructura.

Notese que si se desprecia esta hipotesis se hace necesario conocer la configuración deformada antes de poder plantear las ecuaciones de equilibrio, pero a la vez para poder conocer la configuración deformada es necesario cargar el sistema y resolver las ecuaciones de equilibrio. En consecuencia el problema es no líneal. Este hecho es evidente si se considera la matriz de rígidez local de un elemento cercha:

$$
\begin{Bmatrix}f_1\\f_2\end{Bmatrix} = \frac{AE}{l}
\begin{bmatrix}
1&-1\\
-1&1\end{bmatrix}
\begin{Bmatrix}u_1\\u_2\end{Bmatrix}\, 
$$

de la cual se aprecia que la rigidez depende de la longitud del elemento, la cual depende a la vez de la deformación del mismo.

En este notebook utilizaremos la implementación de los elementos tipo cercha (desarrollada en NBs anteriores) para resolver una estructura considerando de manera aproximada la no linealidad.

En partícular, abordaremos el problema de una cercha de Von Mises (ver figura), sometida a una carga vertical $F$. 

<center>
    <img src="images/cercha_mises.png"
         alt="files"
         style="width:300px">
</center>

Esta cercha tiene la particularidad de que dependiendo del valor de la carga la estructura puede desarrollar un colapso por pandeo subito e invertir su configuración. Para determinar el valor de la carga de colapso es necesario:

* (i) Resolver el problema incrementalmente dividiendo la aplicación de la carga en varios pasos o incrementos $\triangle F$.

* (ii) Considerar el efecto de la deformación en la solución del problema de equilibrio.

## Método de solución

Para resolver el problema se asumirá, en cada incremento, que la rígidez de los elementos, y por ende la rígidez de la estructura, corresponde a la configuración original antes de imponer el incremento de carga. Denotemos esta configuración y su rígidez asociada como ${}^0X$ y ${}^0K$ respectivamente y escribamos la ecuación de equlibrio en esta configuración como:

$$
\left[{}^0K\right]\left\{\triangle U\right\}=\left\{\triangle F\right\}
$$

donde $\triangle U$ sería el desplazamiento producido por $\triangle F$ si es que la rígidez fuera ${}^0K$.

<div class="alert alert-warning">
    
De que depende que las rígideces correspondientes a la configuración original ${}^0K$ y deformada ${}^1K$ sean diferentes?

</div>

Para verificar que tanto se esta violando la condición de equilibrio actualizemos el sistema a la configuración deformada de acuerdo con:


$$
X_i \leftarrow{}^0X+\triangle U
$$

y en la cual $X_i$ denota la aproximación $i$ a la configración deformada.

Recalculemos ahora la rígidez $K=K(X_i)$ en función de la nueva configuración $X_i$ y determinemos también los residuales o fuerzas sin equilibrar dadas por:

$$
R = \triangle F-K(X_i)\triangle U
$$


Una configuración de equilibrio se habrá encontrado cuando $R<tol$ o equivalentemente cuando el sistema

$$
\left[K\right]\left\{\triangle U\right\}=\left\{R\right\}
$$


arroje como resultado un incremento de desplazamiento muy pequeño o tal que $\triangle U<tol$.

gives a null displacement increment such $\triangle U<tol$.

Para verificar esta condición hacemos:

$$
\triangle F \leftarrow R
$$

y resolvemos

$$
\left[K\right]\left\{\triangle U\right\}=\left\{\triangle F\right\}
$$

para determinar el nuevo incremento $\triangle U$.

### Algoritmo 

If $\triangle U < Tol$ Then (estado de equlibrio encontrado):

Almacene la solución:


$$
{}^1X \leftarrow X_i
$$

$$
{}^1U \leftarrow {}^1X - {}^0X
$$

$$
{}^1F \leftarrow  \triangle F
$$


Else

Actualice la configuración:

$$
X_{i+1}\leftarrow{} X_i+\triangle U
$$


$$
U \leftarrow X_{i+1} - {}^0X
$$

Recalcule $K=K(X_{i+1})$


Calcule el residual:

$$
R = \triangle F-K(X_{i+1}) U
$$


y resuelva el sistema:

$$
\left[K\right]\left\{\triangle U\right\}=\left\{R\right\}
$$

para determinar un nuevo incremento $\triangle U$ hasta alcanzar convergencia.

## Solución por elementos finitos

### Subrutinas generales

In [35]:
import numpy as np

In [36]:
def readin():
    nodes = np.loadtxt('files/Cnodes.txt', ndmin=2)
    mats = np.loadtxt('files/Cmater.txt', ndmin=2)
    elements = np.loadtxt('files/Celes.txt', ndmin=2)
    loads = np.loadtxt('files/Cloads.txt', ndmin=2)
    return nodes, mats, elements, loads

In [37]:
def eqcounter(nodes):
    nnodes = nodes.shape[0]
    IBC = np.zeros((nnodes, 2), dtype=np.integer)
    for node in range(nnodes):
        for dof in range(2):
            IBC[node, dof] = int(nodes[node, dof + 3])
    neq = 0
    for node in range(nnodes):
        for dof in range(2):
            if IBC[node, dof] == 0:
                IBC[node, dof] = neq
                neq = neq + 1
    return neq, IBC

In [38]:
def assembly(elements, mats, nodes, neq, DME):
    IELCON = np.zeros((2), dtype=np.integer)
    KG = np.zeros((neq, neq))
    nels = elements.shape[0]
    nnodes = 2
    ndof = 4
    for el in range(nels):
        elcoor = np.zeros((nnodes, 2))
        im = np.int(elements[el , 2])
        par0 = mats[im, 0]
        par1 = mats[im, 1]
        for j in range(nnodes):
            IELCON[j] = elements[el , j+3]
            elcoor[j , 0] = nodes[IELCON[j], 1]
            elcoor[j , 1] = nodes[IELCON[j], 2]
        kloc = ueltruss2D(elcoor, par0, par1)
        dme = DME[el, :ndof]
        for row in range(ndof):
            glob_row = dme[row]
            if glob_row != -1:
                for col in range(ndof):
                    glob_col = dme[col]
                    if glob_col != -1:
                        KG[glob_row, glob_col] = KG[glob_row, glob_col] +\
                                                 kloc[row, col]
    return KG

In [39]:
def ueltruss2D(coord, A, Emod):
    vec = coord[1, :] - coord[0, :]
    length = np.linalg.norm(vec)
    nx = vec[0]/length
    ny = vec[1]/length
    Q = np.array([
        [nx, ny , 0 , 0],
        [0,  0, nx , ny]])
    kl = (A*Emod/length) * np.array([
        [1, -1],
        [-1, 1]])
    kG = Q.T @ kl @ Q
    return kG

In [40]:
def loadasem(loads, IBC, neq, nl):
    RHSG = np.zeros([neq])
    for cont in range(nl):
        il = int(loads[cont, 0])
        ilx = IBC[il , 0]
        ily = IBC[il , 1]
        if ilx != -1:
            RHSG[ilx] = loads[cont, 1]
        if ily != -1:
            RHSG[ily] = loads[cont, 2]
    return RHSG

In [41]:
def DME(nodes, elements):
    nels = elements.shape[0]
    IELCON = np.zeros((nels, 2), dtype=np.integer)
    DME = np.zeros((nels, 4), dtype=np.integer)
    neq, IBC = eqcounter(nodes)
    ndof = 4
    nnodes = 2
    for ele in range(nels):
        for node in range(nnodes):
            IELCON[ele, node] = elements[ele, node + 3]
            glob_num = IELCON[ele, node]
            for loc_num in range(2):
                DME[ele, 2*node + loc_num] = IBC[glob_num, loc_num]
    return DME, IBC, neq

Calcule el operador ensamblador y dejelo listo para iterar

In [8]:
ndof = 4
nnodes = 2
nels = 2
nodes, mats, elements, loads = readin()
neq, IBC = eqcounter(nodes)
R = np.zeros((neq))
F = np.zeros((neq))
UG = np.zeros((neq))
DME, IBC, neq = DME(nodes, elements)

## Iteraciones
A conitnuación se describen de manera completa algunas iteraciones. Para el primer incremento se explica cada paso de la iteración.

### Incremento 1

Asuma estado de equilibrio conocido y dado por: ${}^0X$ , ${}^0F$ and ${}^0U$ .

Calcule $K=K({}^0X)$ para la configuración conicida ${}^0X$, aplique el incremento $\triangle F$ y resuelva:

$$
\left[K({}^0X)\right]\left\{\triangle U\right\}=\left\{\triangle F\right\}
$$

para determinar el incremento $\triangle U$.

In [25]:
ini_x = nodes[2 , 2 ]
KG    = assembly(elements, mats, nodes, neq, DME)
RHSG  = loadasem(loads, IBC, neq, 1)
delu  = np.linalg.solve(KG, RHSG)
print(delu)

[ 0.         -0.00120508]


Actualice:

$$
X_i \leftarrow{}^0X+\triangle U
$$

Recalcule $K=K(X_i)$

y calcue el residual:

$$
R = \triangle F-K(X_i)\triangle U
$$

In [26]:
nodes[2 , 2 ]  = ini_x + delu[1]
UG[1] = nodes[2 , 2 ] - ini_x
KG = assembly(elements, mats, nodes, neq, DME)
R = RHSG-KG.dot(UG)
print(R)

[ 0.00000000e+00 -5.61374779e-05]


Haga:

$$
\triangle F \leftarrow R
$$

y resuelva:

$$
\left[K\right]\left\{\triangle U\right\}=\left\{\triangle F\right\}
$$

para el nuevo incremento $\triangle U$.

In [27]:
loads[0 , 2] = R[1]
RHSG = loadasem(loads, IBC, neq, 1)
delu = np.linalg.solve(KG , RHSG)
print(delu)

[ 0.00000000e+00 -1.73771202e-06]


If $\triangle U < Tol$ Then (equilibrio):

Alamacene la solución:


$$
{}^1X \leftarrow X_i
$$

$$
{}^1U \leftarrow {}^1X - {}^0X
$$

$$
{}^1F \leftarrow  \triangle F
$$


Else

Actualice:

$$
X_{i+1}\leftarrow{} X_i+\triangle U
$$


$$
U \leftarrow X_{i+1} - {}^0X
$$

Recalcule $K=K(X_{i+1})$


Calcule residual:

$$
R = \triangle F-K(X_{i+1}) U
$$


y resuelva:

$$
\left[K\right]\left\{\triangle U\right\}=\left\{R\right\}
$$

para determinar un nuevo incremento $\triangle U$ hasta alcanzar convergencia.

In [28]:
nodes[2 , 2 ]  = nodes[2 , 2 ] + delu[1]
print(nodes[2 , 2 ])
UG[1] = nodes[2 , 2 ] - ini_x
KG = assembly(elements, mats, nodes, neq, DME)
R = RHSG-KG.dot(delu)
print(R)
#
loads[0 , 2] = R[1]
RHSG = loadasem(loads, IBC, neq, 1)
delu = np.linalg.solve(KG , RHSG)
print(delu)

0.9514968869399463
[ 0.00000000e+00 -1.16797104e-10]
[ 0.00000000e+00 -3.61541276e-12]


In [29]:
nodes[2 , 2 ]  = nodes[2 , 2 ]
UG[1] = nodes[2 , 2 ] - ini_x
print(UG)
print(nodes[2 , 2 ])

[ 0.         -0.00120682]
0.9514968869399463


### Incremento 2

In [30]:
ini_x = nodes[2 , 2 ]
KG    = assembly(elements, mats, nodes, neq, DME)
loads = np.loadtxt('files/Cloads.txt', ndmin=2)
RHSG  = loadasem(loads, IBC, neq, 1)
delu  = np.linalg.solve(KG, RHSG)
print(delu)

[ 0.         -0.21915888]


In [15]:
nodes[2 , 2 ]  = ini_x + delu[1]
UG[1] = nodes[2 , 2 ] - ini_x
KG = assembly(elements, mats, nodes, neq, DME)
R = RHSG-KG.dot(UG)
print(R)

[ 0.         -0.65474157]


In [16]:
loads[0 , 2] = R[1]
RHSG = loadasem(loads, IBC, neq, 1)
delu = np.linalg.solve(KG , RHSG)
print(delu)

[ 0.         -0.01634449]


In [17]:
nodes[2 , 2 ]  = nodes[2 , 2 ] + delu[1]
print(nodes[2 , 2 ])
UG[1] = nodes[2 , 2 ] - ini_x
KG = assembly(elements, mats, nodes, neq, DME)
R = RHSG-KG.dot(delu)
print(R)
#
loads[0 , 2] = R[1]
RHSG = loadasem(loads, IBC, neq, 1)
delu = np.linalg.solve(KG , RHSG)
print(delu)

1.1636105543819666
[ 0.        -0.0078351]
[ 0.         -0.00019796]


In [18]:
nodes[2 , 2 ]  = nodes[2 , 2 ]
UG[1] = nodes[2 , 2 ] - ini_x
print(UG)
print(nodes[2 , 2 ])

[ 0.         -0.17673999]
1.1636105543819666


### Incremento 3

In [31]:
ini_x = nodes[2 , 2 ]
KG    = assembly(elements, mats, nodes, neq, DME)
loads = np.loadtxt('files/Cloads.txt', ndmin=2)
RHSG  = loadasem(loads, IBC, neq, 1)
delu  = np.linalg.solve(KG, RHSG)
print(delu)

[ 0.         -0.21915888]


In [32]:
nodes[2 , 2 ]  = ini_x + delu[1]
UG[1] = nodes[2 , 2 ] - ini_x
KG = assembly(elements, mats, nodes, neq, DME)
R = RHSG-KG.dot(UG)
print(R)

[ 0.         -2.02564235]


In [33]:
loads[0 , 2] = R[1]
RHSG = loadasem(loads, IBC, neq, 1)
delu = np.linalg.solve(KG , RHSG)
print(delu)

[ 0.         -0.08783263]


In [34]:
nodes[2 , 2 ]  = nodes[2 , 2 ] + delu[1]
print(nodes[2 , 2 ])
UG[1] = nodes[2 , 2 ] - ini_x
KG = assembly(elements, mats, nodes, neq, DME)
R = RHSG-KG.dot(delu)
print(R)
#
loads[0 , 2] = R[1]
RHSG = loadasem(loads, IBC, neq, 1)
delu = np.linalg.solve(KG , RHSG)
print(delu)

0.6445053822176532
[ 0.        -0.3487758]
[ 0.         -0.01826853]


In [23]:
nodes[2 , 2 ]  = nodes[2 , 2 ]
UG[1] = nodes[2 , 2 ] - ini_x
print(UG)
print(nodes[2 , 2 ])

[ 0.         -0.21090685]
0.9527037093117487


## Problema

<div class="alert alert-warning">
    
 Implemente un algoritmo que realice de manera automatica los pasos de cada incremento y aplique el mismo para determinar al menos 5 configuraciones equilibradas.
</div>


## Formato del notebook

La siguiente celda cambia el formato del Notebook.

In [24]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./nb_style.css', 'r').read()
    return HTML(styles)
css_styling()